In [1]:
import pandas as pd
import inflection
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import datetime
from tabulate import tabulate
from IPython.display import Image
from scipy import stats as ss
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
import xgboost as xgb
import random 
import pickle
import requests
import json

### 0.2 Loading data

In [2]:
df_sales_raw = pd.read_csv(r'C:\Users\laism\Downloads\rossmann-store-sales\train.csv', low_memory=False)

df_store_raw = pd.read_csv(r'C:\Users\laism\Downloads\rossmann-store-sales\store.csv', low_memory=False)

## 10.1 Rossmann Class

In [ ]:
class Rossmann(object):
    def __init__(self):
        #carregando os parametros
        self.competition_distance_scaler = pickle.load(open('parameter/competition_distance_scaler.pkl', 'rb'))
        self.promo_time_week_scaler.pkl = pickle.load(open('parameter/promo_time_week_scaler.pkl.pkl', 'rb'))
        self.competition_time_month_scaler = pickle.load(open('parameter/competition_time_month_scaler.pkl', 'rb'))
        self.year_scaler = pickle.load(open('parameter/year_scaler.pkl', 'rb'))
        self.store_type_scaler = pickle.load(open('parameter/store_type_scaler.pkl', 'rb'))

        state = 1
        
    def data_cleaning(self, df1):

        #renomeando as colunas
        cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
                   'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
                   'CompetitionDistance', 'CompetitionOpenSinceMonth',
                   'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
                   'Promo2SinceYear', 'PromoInterval']

        #função que deixa tudo minusculo e separado por _
        snakecase = lambda x: inflection.underscore(x)
        cols_new = list( map(snakecase, cols_old))

        df1.columns = cols_new

        #mudando o tipo da data
        df1['date'] = pd.to_datetime(df1['date'])

        #preenchendo os NA

        #CompetitionDistance - distance in meters to the nearest competitor store
        #vou assumir que se o valor é mt maior que a distância máxima que tem um competidor proximo então é a msm coisa que dizer que não tem um competidor proxim

        max_value = df1['competition_distance'].max()
        #isnan avalia se é NA
        df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan(x) else x)

        #CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
        #vou assumir que é vazio pqe não tem um competidor mais proximo ou tem um competidor proximo mas não temos info de quando abriu
        #vou substituir pelo mês da data da venda

        df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

        df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

        #Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
        #vou substituir pela data da venda

        df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

        df1['promo2_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

        #PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store
        #dict para trocar o num pelo nome do mês
        month_map = {1: 'Jan',
                     2: 'Fev',
                     3: 'Mar',
                     4: 'Apr',
                     5: 'May',
                     6: 'Jun',
                     7: 'Jul',
                     8: 'Aug',
                     9: 'Sept',
                     10: 'Oct',
                     11: 'Nov',
                     12: 'Dec'}

        #susbtitui o N/A por zero
        df1['promo_interval'].fillna(0, inplace=True)

        #extraindo o mês da data e aplicando o dicionario para fazer a tradução
        df1['month_map'] = df1['date'].dt.month.map(month_map)

        #avaliação se o month_map está dentro do intervalo para ver se a loja está na promoção (1) ou não (0)
        df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

        #mudando os tipos de dados
        #passando para inteiro
        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)

        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

        df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)

        df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)
        
        return df1
    
    def feature_engineering(self, df2):
    
        #year
        df2['year'] = df2['date'].dt.year

        #month
        df2['month'] = df2['date'].dt.month

        #day
        df2['day'] = df2['date'].dt.day

        #week of year
        df2['week_of_wear'] = df2['date'].dt.weekofyear

        #year week
        df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

        #cometition since
        #juntando as duas informações de data (ano e mês) em uma coluna só
        #datetime.datetime() é uma função que monta uma data apartir de valores
        df2['competion_since'] = df2.apply( lambda x: datetime.datetime(year = x['competition_open_since_year'], month = x['competition_open_since_month'], day=1), axis=1)
        df2['competion_time_month'] = ((df2['date'] - df2['competion_since'])/30 ).apply(lambda x: x.days).astype(int)

        #promo since
        df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)

        #trocando caracter para o tipo data
        df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))

        df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

        #assortment
        #Assortment - describes an assortment level: a = basic, b = extra, c = extended
        df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

        #state holiday
        #StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. 
        #a = public holiday, b = Easter holiday, c = Christmas, 0 = None

        df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')

        ######filtragem de variáveis
        #filtrando linhas
        #só vou usar as vendas em que open é diferente de zero
        #sellers tem que ser maior que zero
        df2 = df2[(df2['open'] != 0) & (df2['sales'] > 0)]
        
        #seleção das colunas 
        #baseado no contexto, não vamos ter a coluna customers no momento da previsão pqe não tem como prever quantos customers vai ter
        #promo interval foi derivada e month map é uma coluna auxiliar
        #a coluna open como foi filtrada só tem valor 1
        cols_drop = ['customers', 'open','promo_interval','month_map']
        df2 = df2.drop(cols_drop, axis=1)
        
        return df2
    
    def data_preparation(self, df5):
        ## 5.2 Rescaling
        #Técnica Robust Scaler
        #criando a msm variavel mas em uma nova escala

        #competition_distance
        df5['competition_distance'] = self.competition_distance_scaler.fit_transform(df5[['competition_distance']].values)

        #competition time month
        df5['competition_time_month'] = self.competition_time_month_scaler.fit_transform(df5[['competition_time_month']].values)

        #Técnica Min-Max Scaler:
        #promo time week
        df5['promo_time_week'] = self.promo_time_week_scaler.fit_transform(df5[['promo_time_week']].values)

        #year
        df5['year'] = self.year_scaler.fit_transform(df5[['year']].values)

        ## 5.3 Transformação
        ### 5.3.1 Encoding
        #variaveis categoricas: state_holiday , store_type e assortment

        #state_holiday - One Hot Encoding
        #cria uma coluna pra cada tipo colocando 1 ou 0
        df5 = pd.get_dummies( df5, prefix=['state_holiday'], columns = ['state_holiday'])

        #store_type - Label Encoding
        #substitui a categoria por número
        df5['store_type'] = self.store_type_scaler.fit_transform(df5['store_type'])

        #assortment - Ordinal Encoding
        #substitui a categoria por número de forma ordenada
        assortment_dict = {'basic': 1,
                           'extra': 2,
                           'extended': 3}
        df5['assortment'] = df5['assortment'].map(assortment_dict)

        ### 5.3.3 Nature Transformation

        #separando quais variáveis tem natureza ciclica 
        #month
        df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * (2. * np.pi/12)))
        df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * (2. * np.pi/12)))

        #day
        df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * (2. * np.pi/30)))
        df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * (2. * np.pi/30)))

        #week of year
        df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * (2. * np.pi/52)))
        df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * (2. * np.pi/52)))

        #day_of_week
        df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * (2. * np.pi/7)))
        df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * (2. * np.pi/7)))

        cols_selected = ['store', 'promo', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month', 'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 'competition_time_month', 'promo_time_week', 'month_cos', 'month_sin', 'day_sin', 'day_cos', 'week_of_year_sin', 'week_of_year_cos', 'day_of_week_sin', 'day_of_week_cos']
        
        return df5[cols_selected]


## 10.2 API Handler

In [ ]:
import pickle 
import pandas as pd
from flask import Flask, request
from Rossman import Rossmann

#carregando o modelo
model = pickle.load( open(r'model_rossmann.pkl', 'rb'))

#inicializando a API
app = Flask(__name__)

#criando o end point
#POST é o metodo que envia dados 
@app.route('/rossmann/predict', methods=['POST'])

#quando o end point recebe uma chamada vida post, ele executa a primeira função abaixo dele

def rossmann_predict():
    test_json = request.get_json()
    
    if test_json: #se tem dado, converte json em dataframe
        if isinstance(test_json, dict): #exemplo unico
            test_raw = pd.DataFrame(test_json, index=[0])
        else: #exemplo multiplos
            test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())
        
        #instanciando a classe rossmann
        pipeline = Rossmann()
        
        #data cleaning
        df1 = pipeline.data_cleaning(test_raw)
        
        #feature enginnering
        df2 = pipeline.feature_engineering(df1)
        
        #data preparation
        df3 = pipeline.data_preparation(df2)
        
        #prediction
        df_response = pipeline.Get_prediction(model, test_raw, df3)
        
        return df_response

        
    else: #se não tem dado
        return Response('{}', status=200, mimetype='application/json')

if __name__ == '__main__':
    #'0.0.0.0' end point do local host
    app.run('0.0.0.0')

## 10.3 API Tester

In [1]:
import requests

In [ ]:
#carregando dados de teste
df10 = pd.read_csv(r'C:\Users\laism\Downloads\rossmann-store-sales\test.csv')

In [ ]:
#merge do dataset testes + store
df_test = pd.merge(df10, df_store_raw, how='left', on = 'Store')

#escolhendo a loja para predição
#df_test = df_test[df_test['Store'].isin([24])]
df_test = df_test[df_test['Store'] == 22]

#remove dias fechados
df_test = df_test[df_test['Open'] != 0]
#removendo lojas vazias
df_test = df_test[~df_test['Open'].isnull()]
#apagando a coluna id
df_test = df_test.drop('Id', axis=1)

#convertendo df para Json
import json
data = json.dumps(df_test.to_dict(orient = 'records'))

In [ ]:
# API Call
#url é o end point, pra onde eu vou enviar o pedido
url = 'http://192.168.0.214:5000/rossmann/predict'
#url = 'http://0.0.0.0:5000/rossmann/predict'

#header indica para ip que tipo de dado ela está recebendo
header = { 'Content-type': 'application/json'}

data = data
#post é um metodo para enviar dados
r = requests.post(url, data=data, headers=header)
print('Status Code {}'.format(r.status_code))